In [ ]:
import sys; sys.path.append('..')

In [ ]:
from itertools import combinations
import pickle
from collections import defaultdict
from fractions import Fraction
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from util import step_plot
import networkx as nx
from scipy.stats import pearsonr
import seaborn as sns

In [ ]:
p_incident = Path('../data/incident')
p_core = Path('../data/core')
p_CE = Path('../data/clique_expansion')
p_CE.mkdir(exist_ok=True)
p_dumps = Path('dumps')
p_dumps.mkdir(exist_ok=True)
p_figures = Path('figures')
p_figures.mkdir(exist_ok=True)

plt.rcParams.update({'font.size': 30, 'text.usetex': False, 'lines.linewidth': 7})
for ds_name in ds_names:
    p_incident_ds = p_incident / ds_name
    p_CE_ds = p_CE / ds_name
    p_CE_ds.mkdir(exist_ok=True)
    with (p_incident_ds / 'i2edges.pkl').open('rb') as f:
        i2edges = pickle.load(f)
    with (p_incident_ds / 'v2edges.pkl').open('rb') as f:
        v2edges = pickle.load(f)
    vv2co_deg = defaultdict(int)
    edges_CE = []
    for e in tqdm(list(i2edges.values())):
        for v_1, v_2 in combinations(e, 2):
            if v_1 > v_2:
                v_1, v_2 = v_2, v_1
            vv2co_deg[(v_1, v_2)] += 1
            edges_CE.append((v_1, v_2))
    CE_U = nx.Graph(vv2co_deg.keys())
    print('Clique expansion (unweighted) done!')
    CE_W = nx.MultiGraph(edges_CE)
    print('Clique expansion (weighted) done!')
    with (p_CE_ds / 'CE_U.pkl').open('wb') as f:
        pickle.dump(CE_U, f)
    with (p_CE_ds / 'CE_W.pkl').open('wb') as f:
        pickle.dump(CE_W, f)
    v2cU = nx.core_number(CE_U)
    print('Coreness of clique expansion (unweighted) done!')
    v2cW = dict()
    k = 1
    while n_core := CE_W.number_of_nodes():
        print('{}'.format(n_core), end='\r')
        removed = False
        for v in list(CE_W.nodes):
            if CE_W.degree[v] <= k:
                v2cW[v] = k
                CE_W.remove_node(v)
                removed = True
        if not removed:
            k = min(dict(CE_W.degree()).values())
    print('Coreness of clique expansion (weighted) done!')
    with (p_CE_ds / 'v2cU.pkl').open('wb') as f:
        pickle.dump(v2cU, f)
    with (p_CE_ds / 'v2cW.pkl').open('wb') as f:
        pickle.dump(v2cW, f)
    p_core_ds = p_core / ds_name
    nodes_sorted = sorted(v2edges)
    degree_sorted = [len(v2edges[v]) for v in nodes_sorted]
    cU_sorted = [v2cU[v] for v in nodes_sorted]
    cW_sorted = [v2cW[v] for v in nodes_sorted]
    t2sim_deg = dict()
    t2sim_cU = dict()
    t2sim_cW = dict()
    for c_t in tqdm(list(p_core_ds.glob('c*.pkl'))):
        t = Fraction(c_t.name[2:-4].replace('-', '/'))
        with c_t.open('rb') as f:
            v2c_t = pickle.load(f)
        c_t_sorted = [v2c_t[v] for v in nodes_sorted]
        sim_deg, _ = pearsonr(c_t_sorted, degree_sorted)
        sim_cU, _ = pearsonr(c_t_sorted, cU_sorted)
        sim_cW, _ = pearsonr(c_t_sorted, cW_sorted)
        t2sim_deg[t] = sim_deg
        t2sim_cU[t] = sim_cU
        t2sim_cW[t] = sim_cW
    p_dumps_ds = p_dumps / ds_name
    p_dumps_ds.mkdir(exist_ok=True)
    with (p_dumps_ds / 't2sim_deg.pkl').open('wb') as f:
        pickle.dump(t2sim_deg, f)
    with (p_dumps_ds / 't2sim_cU.pkl').open('wb') as f:
        pickle.dump(t2sim_cU, f)
    with (p_dumps_ds / 't2sim_cW.pkl').open('wb') as f:
        pickle.dump(t2sim_cW, f)

    p_figures_ds = p_figures / ds_name
    p_figures_ds.mkdir(exist_ok=True)
    plt.clf()
    step_plot(t2sim_deg)
    step_plot(t2sim_cU)
    step_plot(t2sim_cW)
    plt.xlabel('t')
    plt.ylabel('Pearson\'s r\nw/ t-hypercoreness')
    plt.savefig(p_figures_ds / 'sim_with.png', bbox_inches='tight')
    plt.savefig(p_figures_ds / 'sim_with.pdf', bbox_inches='tight')
    plt.show()

In [ ]:
plt.rcParams.update({'font.size': 30, 'text.usetex': False, 'lines.linewidth': 7})
t_samples = np.linspace(0.005, 0.995, 100)
for ds_name in ds_names:
    p_core_ds = p_core / ds_name
    possible_t = [Fraction(c_t.name[2:-4].replace('-', '/')) for c_t in p_core_ds.glob('c*.pkl')]
    t_chosen = [min(t for t in possible_t if t >= t_sample) for t_sample in t_samples]
    c_t_lists = []
    nodes = None
    for i_t, t in tqdm(list(enumerate(t_chosen))):
        with (p_core_ds / 'c_{}.pkl'.format(str(t).replace('/', '-'))).open('rb') as f:
            v2c_t = pickle.load(f)
        if i_t == 0:
            nodes = sorted(v2c_t)
        c_t_lists.append([v2c_t[v] for v in nodes])
    sim_among_matrix = np.ones((100, 100))
    for i_1, i_2 in tqdm(list(combinations(range(100), 2))):
        c_t_list_1 = c_t_lists[i_1]
        c_t_list_2 = c_t_lists[i_2]
        r, _ = pearsonr(c_t_list_1, c_t_list_2)
        sim_among_matrix[i_1][i_2] = sim_among_matrix[i_2][i_1] = r

    p_dumps_ds = p_dumps / ds_name
    with (p_dumps_ds / 'sim_among_matrix.pkl').open('wb') as f:
        pickle.dump(sim_among_matrix, f)

    p_figures_ds = p_figures / ds_name
    plt.clf()
    ax = sns.heatmap(sim_among_matrix, xticklabels=False, yticklabels=False, cmap='RdBu')
    plt.xticks([0, 20, 40, 60, 80, 100], ['0', '0.2', '0.4', '0.6', '0.8', '1'])
    plt.yticks([0, 20, 40, 60, 80, 100], ['0', '0.2', '0.4', '0.6', '0.8', '1'])
    plt.xlabel('t')
    plt.savefig(p_figures_ds / 'sim_among.png', bbox_inches='tight')
    plt.savefig(p_figures_ds / 'sim_among.pdf', bbox_inches='tight')
    plt.show()